In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Transformming Feature Engineering Data

In [0]:
forest_mod_trans = pd.DataFrame()
forest_mod_trans['Elevation'] = forest_mod['Elevation']
forest_mod_trans['Aspect'] = forest_mod['Aspect']
forest_mod_trans['Slope'] = np.cos(forest_mod['Slope'])
forest_mod_trans['Horizontal_Distance_To_Hydrology'] = np.sqrt(forest_mod['Horizontal_Distance_To_Hydrology'])
forest_mod_trans['Vertical_Scaled'] = np.log(forest_mod['Vertical_Scaled']+1)
forest_mod_trans['Horizontal_Distance_To_Roadways'] = np.sqrt(forest_mod['Horizontal_Distance_To_Roadways'])
forest_mod_trans['Hillshade_9am'] = forest_mod['Hillshade_9am']
forest_mod_trans['Hillshade_Noon'] = forest_mod['Hillshade_Noon']
forest_mod_trans['Hillshade_3pm'] = forest_mod['Hillshade_3pm']
forest_mod_trans['Horizontal_Distance_To_Fire_Points'] = np.sqrt(forest_mod['Horizontal_Distance_To_Fire_Points'])
forest_mod_trans['Hydr_Slope'] = np.cos(forest_mod['Hydr_Slope'])
forest_mod_trans['Avg_Horizontal'] = np.sqrt(forest_mod['Avg_Horizontal'])
forest_mod_trans['Fire_Road'] = np.sqrt(forest_mod['Fire_Road'])
forest_mod_trans['Hillshades'] = forest_mod['Hillshades']
forest_mod_trans['Hydr_Dist'] = np.sqrt(forest_mod['Hydr_Dist'])
forest_mod_trans['Fire_Hydr'] = np.sqrt(forest_mod['Fire_Hydr'])
forest_mod_trans['Wilderness'] = forest_mod['Wilderness'].astype('category')
forest_mod_trans['Soil'] = forest_mod['Soil'].astype('category')
forest_mod_trans['CoverType'] = forest_mod['CoverType'].astype('category')

In [0]:
forest_mod_trans.to_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Feat_Add_Trans.csv')

In [0]:
forest_mod_trans = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Combined_CoverType_Feat_Add_Trans.csv')

In [0]:
forest_mod_trans = forest_mod_trans.drop(columns = forest_mod_trans.columns[:1])

In [5]:
forest_mod_trans.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Slope,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Fire_Hydr,Wilderness,Soil,CoverType
0,2596,51,-0.989992,16.062378,5.159055,22.583180,221,232,148,79.240141,1.000000,48.466483,82.395388,200.333333,16.062378,57.170797,1,29,4
1,2590,56,-0.416147,14.560220,5.123964,19.748418,220,235,151,78.898669,-0.050328,47.703948,81.332650,202.000000,14.563135,56.731825,1,29,4
2,2804,139,-0.911130,16.370706,5.476464,56.391489,234,238,135,78.236820,0.482979,56.477134,96.441692,202.333333,16.606319,56.519908,1,12,2
3,2785,155,0.660317,15.556349,5.676754,55.587768,238,238,122,78.809898,0.651471,56.400355,96.441692,199.333333,16.408411,56.802289,1,30,2
4,2595,45,-0.416147,12.369317,5.153292,19.773720,220,234,150,78.562077,0.930699,47.314550,81.012345,201.333333,12.369449,56.236109,1,29,4


In [0]:
forest_mod_trans['Wilderness'] = forest_mod_trans['Wilderness'].astype('category')
forest_mod_trans['Soil'] = forest_mod_trans['Soil'].astype('category')
forest_mod_trans['CoverType'] = forest_mod_trans['CoverType'].astype('category')

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
x = forest_mod_trans.drop(columns= ['CoverType'])
y = forest_mod_trans['CoverType']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

Decision Tree

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [16]:
# model with optimal hyperparameters
dt = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
dt.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=25, splitter='best')

In [0]:
y_testpred = dt.predict(x_test)

In [18]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_train,dt.predict(x_train)))

Training Accuracy for DecisionTree Classifier:  0.9333993921929247


In [19]:
print('Training Accuracy for DecisionTree Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for DecisionTree Classifier:  0.9075982192032311


In [20]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.90      0.90      0.90     63514
           2       0.92      0.92      0.92     85045
           3       0.90      0.92      0.91     10747
           4       0.89      0.85      0.87     14998

    accuracy                           0.91    174304
   macro avg       0.90      0.90      0.90    174304
weighted avg       0.91      0.91      0.91    174304



In [21]:
print(confusion_matrix(y_test,y_testpred))

[[57142  6019     7   346]
 [ 5471 78461   371   742]
 [    1   294  9895   557]
 [  592   957   749 12700]]


In [22]:
pd.DataFrame({'columns':x.columns,'Importance':dt.feature_importances_})

,columns,Importance
0,Elevation,0.230321
1,Aspect,0.014613
2,Slope,0.000765
3,Horizontal_Distance_To_Hydrology,0.014822
4,Vertical_Scaled,0.025625
5,Horizontal_Distance_To_Roadways,0.080634
6,Hillshade_9am,0.023185
7,Hillshade_Noon,0.019187
8,Hillshade_3pm,0.012390
9,Horizontal_Distance_To_Fire_Points,0.059705


### Random Forest using decision trees parameters

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [24]:
# model with optimal hyperparameters
rforest = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
rforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=25, verbose=0,
                       warm_start=False)

In [0]:
y_testpred = rforest.predict(x_test)

In [26]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_train,rforest.predict(x_train)))

Training Accuracy for Random Forest Classifier:  0.9616457015844291


In [27]:
print('Training Accuracy for Random Forest Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Random Forest Classifier:  0.9401792270974848


In [28]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.92      0.94     63514
           2       0.93      0.96      0.95     85045
           3       0.94      0.95      0.94     10747
           4       0.95      0.87      0.91     14998

    accuracy                           0.94    174304
   macro avg       0.94      0.93      0.93    174304
weighted avg       0.94      0.94      0.94    174304



In [29]:
print(confusion_matrix(y_test,y_testpred))

[[58537  4823     0   154]
 [ 2569 82026   177   273]
 [    3   209 10249   286]
 [  454   958   521 13065]]


In [30]:
pd.DataFrame({'columns':x.columns,'Importance':rforest.feature_importances_})

,columns,Importance
0,Elevation,0.257785
1,Aspect,0.023606
2,Slope,0.001528
3,Horizontal_Distance_To_Hydrology,0.022972
4,Vertical_Scaled,0.026884
5,Horizontal_Distance_To_Roadways,0.069115
6,Hillshade_9am,0.022067
7,Hillshade_Noon,0.019377
8,Hillshade_3pm,0.018663
9,Horizontal_Distance_To_Fire_Points,0.053081


### KNN

In [31]:
from sklearn.preprocessing import StandardScaler

x1 = forest_mod_trans.drop(columns= ['CoverType','Soil','Wilderness'])
y = forest_mod_trans['CoverType']

x_std = StandardScaler().fit_transform(x1)

x_std = pd.DataFrame(x_std, columns = x1.columns)

x = pd.concat([x_std, forest_mod_trans[['Soil','Wilderness']]], axis = 1)
x.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Scaled,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Hydr_Slope,Avg_Horizontal,Fire_Road,Hillshades,Hydr_Dist,Fire_Hydr,Soil,Wilderness
0,-1.297805,-0.935157,-1.389008,0.164973,-0.867052,-1.395274,0.330743,0.439143,0.142960,2.587999,1.300680,1.054833,1.068195,0.530152,0.135858,2.574818,29,1
1,-1.319235,-0.890480,-0.578038,-0.056053,-1.014305,-1.567204,0.293388,0.590899,0.221342,2.564187,-0.148505,0.979149,1.008612,0.645367,-0.082435,2.529746,29,1
2,-0.554907,-0.148836,-1.277558,0.210340,0.464875,0.655217,0.816364,0.742654,-0.196691,2.518036,0.587323,1.849922,1.855709,0.668410,0.215057,2.507987,12,1
3,-0.622768,-0.005869,0.943243,0.090516,1.305345,0.606471,0.965786,0.742654,-0.536343,2.557997,0.819799,1.842302,1.855709,0.461022,0.186241,2.536981,30,1
4,-1.301377,-0.988770,-0.578038,-0.378421,-0.891238,-1.565670,0.293388,0.540313,0.195215,2.540716,1.205062,0.940499,0.990654,0.599281,-0.401841,2.478848,29,1


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [34]:
knn = KNeighborsClassifier(metric = 'cityblock', n_neighbors= 4, weights= 'distance')
knn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cityblock',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='distance')

In [0]:
y_testpred = knn.predict(x_test)

In [36]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_train,knn.predict(x_train)))

Training Accuracy for KNN Classifier:  1.0


In [37]:
print('Training Accuracy for KNN Classifier: ', accuracy_score(y_test,y_testpred))

Training Accuracy for KNN Classifier:  0.9231515054158252


In [38]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.93      0.92      0.92     63514
           2       0.93      0.94      0.93     85045
           3       0.91      0.91      0.91     10747
           4       0.88      0.87      0.88     14998

    accuracy                           0.92    174304
   macro avg       0.91      0.91      0.91    174304
weighted avg       0.92      0.92      0.92    174304



In [39]:
print(confusion_matrix(y_test,y_testpred))

[[58184  4946     7   377]
 [ 4143 79873   319   710]
 [    9   281  9806   651]
 [  491   828   633 13046]]


### Ensemble

In [0]:
#base_LR = LogisticRegression(random_state = 25)
base_DT = DecisionTreeClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
#base_NB = GaussianNB()
base_RF = RandomForestClassifier(criterion = "entropy", 
                                  random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10)
base_KNN = KNeighborsClassifier(metric = 'cityblock', 
                                n_neighbors= 4, 
                                weights= 'distance')

In [0]:
x = forest_mod_trans.drop(columns= ['CoverType'])
y = forest_mod_trans['CoverType']

### Bagging Classifier

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .3, random_state=25)

In [0]:
from sklearn.ensemble import BaggingClassifier

Bagged DT

In [45]:
bgcl_DT = BaggingClassifier(base_estimator=base_DT, n_estimators = 51, oob_score=True, random_state=25)

bgcl_DT.fit(x_train, y_train)



BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=25,

In [0]:
y_testpred = bgcl_DT.predict(x_test)

In [47]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_train,bgcl_DT.predict(x_train)))

Training Accuracy for Bagged DT:  0.9663984972019237


In [48]:
print('Training Accuracy for Bagged DT: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged DT:  0.945021342023132


In [49]:
print('OOB Score for Bagged DT:', bgcl_DT.oob_score_)

OOB Score for Bagged DT: 0.9415649556930279


In [50]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.95      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.96      0.95     10747
           4       0.94      0.89      0.92     14998

    accuracy                           0.95    174304
   macro avg       0.94      0.94      0.94    174304
weighted avg       0.95      0.95      0.94    174304



In [51]:
print(confusion_matrix(y_test,y_testpred))

[[59177  4147     3   187]
 [ 2637 81921   172   315]
 [    1   178 10271   297]
 [  417   777   452 13352]]


Bagged RF

In [52]:
bgcl_RF = BaggingClassifier(base_estimator=base_RF, n_estimators = 51, oob_score=True, random_state=25)
bgcl_RF.fit(x_train, y_train)

BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='entropy',
                                                        max_depth=19,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=10,
                                                        min_samples_split=10,
                                                        min_weight_fraction_leaf=0.0,
  

In [0]:
y_testpred = bgcl_RF.predict(x_test)

In [54]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_train,bgcl_RF.predict(x_train)))

Training Accuracy for Bagged RF:  0.9460546632965174


In [55]:
print('Training Accuracy for Bagged RF: ', accuracy_score(y_test,y_testpred))

Training Accuracy for Bagged RF:  0.9279018266935928


In [56]:
print('OOB Score for Bagged RF:', bgcl_RF.oob_score_)

OOB Score for Bagged RF: 0.9272623110437955


In [57]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.94      0.91      0.92     63514
           2       0.92      0.96      0.94     85045
           3       0.92      0.94      0.93     10747
           4       0.94      0.83      0.88     14998

    accuracy                           0.93    174304
   macro avg       0.93      0.91      0.92    174304
weighted avg       0.93      0.93      0.93    174304



In [58]:
print(confusion_matrix(y_test,y_testpred))

[[57638  5699     1   176]
 [ 3086 81463   219   277]
 [    3   287 10118   339]
 [  608  1247   625 12518]]


Bagged KNN

In [18]:
bgcl_KNN = BaggingClassifier(base_estimator=base_KNN, n_estimators = 51, oob_score=True, random_state=25)
bgcl_KNN.fit(x_train, y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='cityblock',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=4, p=2,
                                                      weights='distance'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=1.0, n_estimators=51, n_jobs=None, oob_score=True,
                  random_state=25, verbose=0, warm_start=False)

In [0]:
y_testpred = bgcl_KNN.predict(x_test)

In [20]:
print('Training Accuracy for Bagged KNN: ', accuracy_score(y_train,bgcl_KNN.predict(x_train)))

Training Accuracy for Bagged KNN:  0.9998377214119221


In [21]:
print('Testing Accuracy for Bagged KNN: ', accuracy_score(y_test,y_testpred))

Testing Accuracy for Bagged KNN:  0.9280968881953369


In [22]:
print('OOB Score for Bagged KNN:', bgcl_KNN.oob_score_)

OOB Score for Bagged KNN: 0.9266500781887742


In [23]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.93      0.92      0.93     63514
           2       0.93      0.95      0.94     85045
           3       0.91      0.91      0.91     10747
           4       0.89      0.86      0.88     14998

    accuracy                           0.93    174304
   macro avg       0.92      0.91      0.91    174304
weighted avg       0.93      0.93      0.93    174304



In [24]:
print(confusion_matrix(y_test,y_testpred))

[[58494  4711     3   306]
 [ 3666 80530   276   573]
 [    5   278  9806   658]
 [  486   872   699 12941]]


### AdaBoost

In [0]:
from sklearn.ensemble import AdaBoostClassifier

AdaBoost DT

In [0]:
abcl_DT = AdaBoostClassifier(base_estimator=base_DT,n_estimators=50,random_state=25)

In [27]:
abcl_DT.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                    

In [0]:
y_testpred = abcl_DT.predict(x_test)

In [29]:
print('Training Accuracy for AdaBoost DT: ', accuracy_score(y_train,abcl_DT.predict(x_train)))

Training Accuracy for AdaBoost DT:  0.9999975412335139


In [30]:
print('Testing Accuracy for AdaBoost DT: ', accuracy_score(y_test,y_testpred))

Testing Accuracy for AdaBoost DT:  0.9408676794565816


In [31]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.94      0.93      0.94     63514
           2       0.94      0.96      0.95     85045
           3       0.94      0.95      0.94     10747
           4       0.95      0.87      0.91     14998

    accuracy                           0.94    174304
   macro avg       0.94      0.93      0.93    174304
weighted avg       0.94      0.94      0.94    174304



In [32]:
print(confusion_matrix(y_test,y_testpred))

[[59033  4384     0    97]
 [ 2954 81731   166   194]
 [    2   210 10187   348]
 [  493   981   478 13046]]


AdaBoost RF

In [0]:
abcl_RF = AdaBoostClassifier(base_estimator=base_RF,n_estimators=50,random_state=25)

In [19]:
abcl_RF.fit(x_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=19,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=10,
                                                         min_samples_split=10,
                                    

In [0]:
y_testpred = abcl_RF.predict(x_test)

In [21]:
print('Training Accuracy for AdaBoost RF: ', accuracy_score(y_train,abcl_RF.predict(x_train)))

Training Accuracy for AdaBoost RF:  0.919667181368451


In [22]:
print('Testing Accuracy for AdaBoost RF: ', accuracy_score(y_test,y_testpred))

Testing Accuracy for AdaBoost RF:  0.8790733431246558


In [23]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.89      0.81      0.85     63514
           2       0.86      0.95      0.90     85045
           3       0.91      0.94      0.92     10747
           4       0.94      0.75      0.83     14998

    accuracy                           0.88    174304
   macro avg       0.90      0.86      0.88    174304
weighted avg       0.88      0.88      0.88    174304



In [24]:
print(confusion_matrix(y_test,y_testpred))

[[51594 11861     0    59]
 [ 4291 80381   184   189]
 [    1   197 10070   479]
 [ 1922  1068   827 11181]]


GradientBoost

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
gbcl = GradientBoostingClassifier(random_state = 25,
                                  max_depth=19, 
                                  min_samples_leaf=10,
                                  min_samples_split=10,
                                  n_estimators = 50)
gbcl.fit(x_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=19,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=10, min_samples_split=10,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=25, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
y_testpred = gbcl.predict(x_test)

In [28]:
print('Training Accuracy for GradientBoost Classifier: ', accuracy_score(y_train, gbcl.predict(x_train)))

Training Accuracy for GradientBoost Classifier:  1.0


In [29]:
print('Testing Accuracy for GradientBoost Classifier: ', accuracy_score(y_test,y_testpred))

Testing Accuracy for GradientBoost Classifier:  0.9735921149256471


In [30]:
print(classification_report(y_test, y_testpred))

              precision    recall  f1-score   support

           1       0.98      0.97      0.97     63514
           2       0.97      0.98      0.98     85045
           3       0.97      0.97      0.97     10747
           4       0.96      0.94      0.95     14998

    accuracy                           0.97    174304
   macro avg       0.97      0.97      0.97    174304
weighted avg       0.97      0.97      0.97    174304



In [31]:
print(confusion_matrix(y_test,y_testpred))

[[61675  1718     2   119]
 [ 1214 83514   103   214]
 [    0   131 10415   201]
 [  210   421   270 14097]]
